In [1]:
ip="localhost"

In [2]:
! curl -X GET localhost:9200/_cat/indices?v

health status index   uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   finger1 TFiTtijLStS6P7rHYwPLOQ   1   1          6            0        4kb            4kb


In [1]:
! curl -X DELETE localhost:9200/*/

{"acknowledged":true}

In [19]:
def request_create_index():
    return  {
            "settings" : {
                "number_of_shards": 1,
                "number_of_replicas": 1,

                    "similarity": {
                      "scripted_tfidf": {
                        "type": "scripted",
                        "script": {
                          "source": "double t_idf = Math.ceil(Math.log((field.docCount+1.0)/(term.docFreq+1.0))); return doc.freq*t_idf;"
                        }
                      }
                    }
                  },
            "mappings": {
                    "properties": {
                        "minutia": {"type":"nested", 
                                    "properties":{
                                        "text": {"type":"text", "similarity":"scripted_tfidf"},
                                        "min_id":{"type":"text"}
                                    }},
                       "id" : {"type":"text"},
                    }

            }
        }

def create_request_minutia(json_text):
    return { "_op_type":"create",
                "_index": "finger",
                "_source": {
                    "id": json_text['id'],
                    "minutia": [{"min_id": i, "text": text} for i, text in enumerate(json_text['minutia'])]
                }
           }

In [20]:
create_request_minutia({"id":1,"minutia":["1_2 32 23 23","122 12 13232"]})

{'_op_type': 'create',
 '_index': 'finger',
 '_source': {'id': 1,
  'minutia': [{'min_id': 0, 'text': '1_2 32 23 23'},
   {'min_id': 1, 'text': '122 12 13232'}]}}

In [22]:
from elasticsearch import Elasticsearch, helpers
import multiprocessing
class Elasticsearch_service():
    
    def __init__(self, ip="localhost", timeout = 1000, index_name='finger' ):
        self.ip = ip
        self.es = Elasticsearch(hosts=ip)
        self.timeout = timeout
        self.index_name = index_name

        
    def create_index(self):
        request_body = request_create_index()
        try:
            ret = self.es.indices.create(index = self.index_name, body = request_body,request_timeout=10000)
            return ret['acknowledged']
        except:
            return False
        
    def indexing(self,list_json_text):
        pool_index = multiprocessing.Pool()
        request_body = pool_index.map(create_request_minutia, list_json_text)
        try:
            
            helpers.bulk(self.es, request_body,request_timeout=self.timeout)
            return True
        except:
            return False

In [18]:
es_s = Elasticsearch_service()

In [28]:
es_s.create_index()

True

In [11]:
from textwrap import wrap


class generate_term:
    def __init__(self, bits):
        self.bits = bits

    def set_bits(self, bits):
        self.bits = bits

    def generate_terms(self, line):
        values_bit = wrap(line.replace(" ", ""), self.bits)
        return ["{}_{}".format(i, int(x1, 2)) for i, x1 in enumerate(values_bit)]

    def create_terms(self, text):
        terms = self.generate_terms(text)
        return " ".join(terms)


In [12]:
g = generate_term(8)

In [13]:
import multiprocessing
pool_index = multiprocessing.Pool()
list_text_min= pool_index.map(g.create_terms,["000101010101011010101", "101011101010101010101"])

In [14]:
list_text_min

['0_21 1_86 2_21', '0_174 1_170 2_21']

In [ ]:

import datetime  
from functools import reduce
from itertools import chain
import multiprocessing
import pandas as pd
import os
import numpy as np
es = Elasticsearch(hosts=ip)

#listNS = [18,16,12]
#listND = [5,5,7]
#list_quality = [0,15,20,30,40]
#list_bits = [8,12,16]


listNS = [16]
listND = [5]
list_quality = [0]
list_bits = [8]
pool_index = multiprocessing.Pool()
pool_insert = multiprocessing.Pool()
hashs = None
for ns,nd in zip(listNS,listND):
        
    for quality in list_quality:

       # os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_index.txt  ../data-fingerprint/SD14/Finger_data/Vector_Index/ {} {} {}". format(ns,nd,quality))
       # os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_search.txt ../data-fingerprint/SD14/Finger_data/Vector_Search/ {} {} {}".format(ns,nd,quality))
        print("    Vetores Gerados")

        for bits in list_bits:

            print("bits:", bits,"n_bits:",ns*ns*(nd+1),"hashs:",ns*ns*(nd+1)//bits, "quality:",quality )




            ###Indexação

            list_text = []

            for x,i,files in os.walk("../data-fingerprint/SD14/Finger_data/Vector_Index/"):
                for ind,file in enumerate(files):
                    if(True):#ind >=2000):
                         
                        if(ind == 500):
                            break


                        print(ind+1,file)
                        file_op = open(x+file)
                        vector_data = file_op.readlines() 
                        file_op.close()
                        list_text_min= pool_index.map(generate_word,vector_data)
                        id = int(file.split('.')[0])
                        len_list = len(list_text_min)
                        list_text += list(gen_command(list_text_min,id))
                       
                        #print(hashs)
                        #print(len(hashs.text_in_hashs))
                        #list(map(generate_word, zip(list_positions, np.repeat(id,len_list), list(range(len_list)))))                          
                        
                            
                            
            print("    Bulk Genereted")
            a = helpers.bulk(es,list_text,request_timeout=1000)

            print("    Indexed")

In [27]:
es = Elasticsearch(hosts=ip)

def gen_command(text,id_):
    for min_,text_ in enumerate(text):
        yield { "_op_type":"create",
                "_index": "finger",
                "_source": {
                    "id": id_,
                    "min": min_,
                    "text":text_
                }
        } 

In [8]:
g = generate_word(8)

In [9]:
g.create_word(" 0 0 0 0 0 0 0 001 1  1 1 1 0 0 0 0 ")

'0_0 1_124 2_0'

In [28]:
from textwrap import wrap

def generate_list_int(tuple_):
    line = tuple_[0]
    bits = tuple_[1]
    values_bit = wrap(line.replace(" ",""),bits)
    #print(len(values_bit))

    return list(map(lambda x1: int(x1,2),values_bit))



def generate_word(text):

    elem = generate_list_int((text,bits))
    text = ''
    #print(elem)
    list_not_found_8bits = [32, 33, 34, 39, 40, 41, 42, 51, 52, 61, 162, 171, 172, 181, 182,183, 184, 189, 190, 191, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 43, 50, 173, 180]

    for i,value in enumerate(elem):
        if(i not in list_not_found_8bits):

            text += '{}:{} '.format(i,value)
    
    
    
    #id = 1
    return text



In [29]:

import datetime  
from functools import reduce
from itertools import chain
import multiprocessing
import pandas as pd
import os
import numpy as np
es = Elasticsearch(hosts=ip)

#listNS = [18,16,12]
#listND = [5,5,7]
#list_quality = [0,15,20,30,40]
#list_bits = [8,12,16]


listNS = [16]
listND = [5]
list_quality = [0]
list_bits = [8]
pool_index = multiprocessing.Pool()
pool_insert = multiprocessing.Pool()
hashs = None
for ns,nd in zip(listNS,listND):
        
    for quality in list_quality:

       # os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_index.txt  ../data-fingerprint/SD14/Finger_data/Vector_Index/ {} {} {}". format(ns,nd,quality))
       # os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_search.txt ../data-fingerprint/SD14/Finger_data/Vector_Search/ {} {} {}".format(ns,nd,quality))
        print("    Vetores Gerados")

        for bits in list_bits:

            print("bits:", bits,"n_bits:",ns*ns*(nd+1),"hashs:",ns*ns*(nd+1)//bits, "quality:",quality )




            ###Indexação

            list_text = []

            for x,i,files in os.walk("../data-fingerprint/SD14/Finger_data/Vector_Index/"):
                for ind,file in enumerate(files):
                    if(True):#ind >=2000):
                         
                        if(ind == 500):
                            break


                        print(ind+1,file)
                        file_op = open(x+file)
                        vector_data = file_op.readlines() 
                        file_op.close()
                        list_text_min= pool_index.map(generate_word,zip(vector_data,list(np.repeat(bits,len(vector_data)))))
                        id = int(file.split('.')[0])
                        len_list = len(list_text_min)
                        list_text += list(gen_command(list_text_min,id))
                       
                        #print(hashs)
                        #print(len(hashs.text_in_hashs))
                        #list(map(generate_word, zip(list_positions, np.repeat(id,len_list), list(range(len_list)))))                          
                        
                            
                            
            print("    Bulk Genereted")
            a = helpers.bulk(es,list_text,request_timeout=1000)

            print("    Indexed")
            
            
        
    




    

    Vetores Gerados
bits: 8 n_bits: 1536 hashs: 192 quality: 0
1 1780.txt
2 1597.txt
3 1742.txt
4 2133.txt
5 1502.txt
6 1798.txt
7 1955.txt
8 1689.txt
9 2139.txt
10 1684.txt
11 1701.txt
12 1910.txt
13 4817.txt
14 2240.txt
15 1779.txt
16 1604.txt
17 1981.txt
18 1990.txt
19 1570.txt
20 1951.txt
21 2054.txt
22 2048.txt
23 1768.txt
24 1771.txt
25 1928.txt
26 1869.txt
27 1603.txt
28 1843.txt
29 1905.txt
30 1892.txt
31 4521.txt
32 1877.txt
33 2056.txt
34 1977.txt
35 1720.txt
36 1696.txt
37 2109.txt
38 4799.txt
39 4868.txt
40 2144.txt
41 1616.txt
42 1641.txt
43 1891.txt
44 1559.txt
45 1864.txt
46 4667.txt
47 5164.txt
48 4823.txt
49 4633.txt
50 1510.txt
51 2216.txt
52 4567.txt
53 1985.txt
54 1791.txt
55 1534.txt
56 1728.txt
57 4664.txt
58 1639.txt
59 5026.txt
60 5027.txt
61 2112.txt
62 1939.txt
63 1605.txt
64 1518.txt
65 2069.txt
66 1569.txt
67 1602.txt
68 2055.txt
69 1972.txt
70 1971.txt
71 1811.txt
72 1761.txt
73 4866.txt
74 1576.txt
75 1880.txt
76 2223.txt
77 1974.txt
78 2071.txt
79 1589.tx

In [60]:
list_text[1]

{'_op_type': 'create',
 '_index': 'finger1',
 '_source': {'id': 1780,
  'min': 1,
  'text': '0:7 1:224 2:31 3:248 4:63 5:252 6:127 7:254 8:127 9:254 10:255 11:255 12:255 13:255 14:255 15:255 16:255 17:255 18:255 19:255 20:255 21:255 22:127 23:254 24:127 25:254 26:63 27:252 28:31 29:248 30:7 31:224 32:0 33:0 34:0 35:68 36:0 37:0 38:0 39:0 40:0 41:0 42:0 43:1 44:24 45:196 46:32 47:0 48:0 49:70 50:0 51:0 52:0 53:35 54:24 55:196 56:0 57:0 58:0 59:70 60:48 61:0 62:0 63:35 64:24 65:132 66:0 67:0 68:0 69:0 70:0 71:32 72:0 73:1 74:16 75:128 76:0 77:0 78:0 79:16 80:0 81:0 82:0 83:0 84:0 85:0 86:0 87:0 88:35 89:24 90:128 91:0 92:0 93:0 94:0 95:0 96:0 97:0 98:35 99:24 100:128 101:0 102:0 103:0 104:0 105:0 106:0 107:0 108:35 109:24 110:128 111:0 112:0 113:0 114:0 115:0 116:0 117:0 118:3 119:24 120:132 121:0 122:0 123:0 124:0 125:0 126:0 127:0 128:35 129:24 130:196 131:0 132:0 133:0 134:0 135:0 136:0 137:0 138:3 139:24 140:132 141:0 142:0 143:0 144:0 145:0 146:0 147:0 148:0 149:6 150:48 151:128 152

In [14]:

def gendata1(list_text):

  #  list_not_index= []

    for i,pos in enumerate(list_text):
        yield {"index": "finger1" }

        yield {"_source":["id"],"size":100,"query": {"match":{"text": pos}} }

                            
                            
                            
                            

In [71]:
s = {"_source":["id"],
  "query": {
    "query_string": {
      "query": "0:7",
      "default_field": "field"
    
    }
  }
}

In [40]:
s = {"_source":["id"], "size":100,
   "query": {
    "match":{
     "text":  list_text_min[0]  
    }
  }
}


In [15]:
list(gendata1([ "2:31 3:248 4:63 5:252 6:127 7:254 8:127 9:254 10:255 11:255 ","2:31 3:248 4:63 5:252 6:127 7:254 8:127 9:254 10:255 11:255 ", "2:31 3:248 4:63 5:252 6:127 7:254 8:127 9:254 10:255 11:255 "]))

[{'index': 'finger1'},
 {'_source': ['id'],
  'size': 100,
  'query': {'match': {'text': '2:31 3:248 4:63 5:252 6:127 7:254 8:127 9:254 10:255 11:255 '}}},
 {'index': 'finger1'},
 {'_source': ['id'],
  'size': 100,
  'query': {'match': {'text': '2:31 3:248 4:63 5:252 6:127 7:254 8:127 9:254 10:255 11:255 '}}},
 {'index': 'finger1'},
 {'_source': ['id'],
  'size': 100,
  'query': {'match': {'text': '2:31 3:248 4:63 5:252 6:127 7:254 8:127 9:254 10:255 11:255 '}}}]

In [47]:
%%time
data_response = es.msearch(body=list(gendata1(list_text_min)),request_timeout=1000)


CPU times: user 2.5 s, sys: 5.07 s, total: 7.57 s
Wall time: 55.4 s


In [46]:
%%time
a = es.search(body=s,index='finger1')

CPU times: user 59 ms, sys: 159 ms, total: 219 ms
Wall time: 2.23 s


In [20]:
a

{'took': 423,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 37.930954,
  'hits': [{'_index': 'finger1',
    '_type': '_doc',
    '_id': '5UjgsHIBTAOKEQx95Ze8',
    '_score': 37.930954,
    '_source': {'id': 1543}},
   {'_index': 'finger1',
    '_type': '_doc',
    '_id': 'gEnhsHIBTAOKEQx9uxp3',
    '_score': 37.65575,
    '_source': {'id': 1651}},
   {'_index': 'finger1',
    '_type': '_doc',
    '_id': 'b0nisHIBTAOKEQx9w7Cr',
    '_score': 37.253746,
    '_source': {'id': 4973}},
   {'_index': 'finger1',
    '_type': '_doc',
    '_id': 'vEnisHIBTAOKEQx9IVf7',
    '_score': 37.128063,
    '_source': {'id': 1517}},
   {'_index': 'finger1',
    '_type': '_doc',
    '_id': 'U0nisHIBTAOKEQx9wK9p',
    '_score': 37.113815,
    '_source': {'id': 2143}},
   {'_index': 'finger1',
    '_type': '_doc',
    '_id': 'e0jgsHIBTAOKEQx9t3v2',
    '_score': 37.073177,
    '_source': {'id'

In [54]:
a = '''{
  "query": { 
    "bool": {
      "should":[{ 
          "bool":{
              "must": [ 
                { "match":  { "_id": "0" }},
                { "match": { "_index": "hash1"}}

              ]}},
          {"bool":
              {"must": [ 
                { "match":  { "_id": "0" }},
                { "match": { "_index": "hash0"}}
              ]
              }
          }
          
        ]
        
    }                                
  }, "size":0,
  "aggs":
      { "value_count" : {
          "terms":{
          "field" : "pos_text" }
          
      } }

  
    

}
'''

In [38]:

import datetime  
from functools import reduce
from itertools import chain
import multiprocessing
import pandas as pd
import os
import numpy as np
from elasticsearch import Elasticsearch,helpers
es = Elasticsearch(hosts=ip)

#listNS = [18,16,12]
#listND = [5,5,7]
#list_quality = [0,15,20,30,40]
#list_bits = [8,12,16]


listNS = [16]
listND = [5]
list_quality = [15]
list_bits = [8]
pool_index = multiprocessing.Pool()
pool_insert = multiprocessing.Pool()
hashs = None
for ns,nd in zip(listNS,listND):
        
    for quality in list_quality:

      #  os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_index.txt  ../data-fingerprint/SD14/Finger_data/Vector_Index/ {} {} {}". format(ns,nd,quality))
       # os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_search.txt ../data-fingerprint/SD14/Finger_data/Vector_Search/ {} {} {}".format(ns,nd,quality))
        print("    Vetores Gerados")

        for bits in list_bits:

            print("bits:", bits,"n_bits:",ns*ns*(nd+1),"hashs:",ns*ns*(nd+1)//bits, "quality:",quality )
  
            ###Search

            pool = multiprocessing.Pool()


            pos_rank =[]
            for x,i,files in os.walk("../data-fingerprint/SD14/Finger_data/Vector_Search/"):
                count = 0
                for file in files:

                    if(True):#count >= 618):

                        file_op = open(x+file)
                        vector_data = file_op.readlines() 
                        file_op.close()
                        time1 = datetime.datetime.now()
                        list_text_min= pool.map(generate_word,zip(vector_data,list(np.repeat(bits,len(vector_data)))))
                        #id = int(file.split('.')[0])
                        #len_list = len(list_text_min)
                        #list_text += list(gen_command(list_text_min,id))
                        print(len(list_text_min)) 
                        data_response = es.msearch(body=list(gendata1(list_text_min)),request_timeout=1000)

                        #a = pool.map(realize_rank,zip(vector_data,np.repeat(bits,len(vector_data)),np.repeat([index_not_found],len(vector_data),axis=0)))
                        
                        #a = map(realize_rank,zip(vector_data,np.repeat(bits,len(vector_data))))

                       # persons = list(chain.from_iterable(a))

                        #ranking = func_rank_elements(persons)
                        time2 = datetime.datetime.now()
                        id = file.split(".")[0]
                        pos_  = -1
                        #for p in range(len(ranking)):
                         #   if(id  == ranking[p]):
                          #      pos_ = p + 1
                           #     break
                        print("count:",count,"filename:",file,"pos_rank:",pos_,"time:",time2-time1)
                        pos_rank.append([pos_,file,time2-time1])
                        
                    count+=1
                    break
                    
            

            df = pd.DataFrame(pos_rank,columns=['pos_rank','file','time'])
           # df.to_csv('Resultado_Preliminar/result_query{}_{}_{}.csv'.format(ns*ns*(nd+1),quality,bits),index=False)


    Vetores Gerados
bits: 8 n_bits: 1536 hashs: 192 quality: 15
126
count: 0 filename: 368.txt pos_rank: -1 time: 0:00:53.855753


'35:0 36:0 37:0 38:0 44:0 45:0 46:0 47:0 48:0 49:0 53:0 54:0 55:0 56:0 57:0 58:0 59:0 60:0 62:0 63:0 64:0 65:0 66:0 67:0 68:0 69:0 70:0 71:0 72:0 73:0 74:0 75:0 76:0 77:0 78:0 79:0 80:0 81:0 82:0 83:0 84:0 85:0 86:0 87:0 88:0 89:0 90:0 91:0 92:0 93:0 94:0 95:0 96:0 97:0 98:0 99:0 100:0 101:0 102:0 103:0 104:0 105:0 106:0 107:0 108:0 109:0 110:0 111:0 112:0 113:0 114:0 115:0 116:0 117:0 118:0 119:0 120:0 121:0 122:0 123:0 124:0 125:0 126:0 127:0 128:0 129:0 130:0 131:0 132:0 133:0 134:0 135:0 136:0 137:0 138:8 139:64 140:0 141:0 142:0 143:0 144:0 145:0 146:0 147:1 148:12 149:98 150:0 151:0 152:0 153:0 154:0 155:0 156:0 157:1 158:12 159:99 160:0 161:0 163:0 164:0 165:0 166:0 167:0 168:8 169:98 170:0 174:0 175:0 176:0 177:0 178:0 179:0 185:0 186:0 187:0 188:0 '

In [ ]:
%%time
import numpy as np
from textwrap import wrap
import os

ns = 16
nd = 5
quality = 0

os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_index.txt  ../data-fingerprint/SD14/Finger_data/Vector_Index/ {} {} {}".format(ns,nd,quality))
os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_search.txt ../data-fingerprint/SD14/Finger_data/Vector_Search/ {} {} {}".format(ns,nd,quality))
       
print("Generate Vectors")
    
    
es = Elasticsearch(hosts=ip)

for x,i,files in os.walk("../data-fingerprint/SD14/Finger_data/Vector_Index/"):
    for ind,file in enumerate(files):
        if(True):#ind >=2000):
            if(ind %300 == 0 and ind != 0):
                list_for_bulk = convert_hash_to_bulk(hashs,hashs_boolean)
                print(len(list_for_bulk))
                tam = len(list_for_bulk)
                #l1 = list_for_bulk[0:10]
                #l2 = list_for_bulk[int(tam/2):tam]
                helpers.bulk(es,list_for_bulk,request_timeout=1000)
                print("Send bulk")

                hashs = Hash(96,16)
                print("Clear Hash")

              #  break

            print(ind+1,file)
            vector_data = sc.textFile(x + file)
            list_positions = vector_data.map(lambda x:x.replace(" ","")).map(lambda x:wrap(x,16)).map(lambda x: list(map(lambda x1: int(x1,2),x)) ).collect()
            id = int(file.split('.')[0])
            len_list = len(list_positions)
            print(len_list)
            list(map(generate_word, list_positions, np.repeat(id,len_list), list(range(len_list))))

            

list_for_bulk = convert_hash_to_bulk(hashs,hashs_boolean)
print(len(list_for_bulk))
tam = len(list_for_bulk)
#l1 = list_for_bulk[0:10]
#l2 = list_for_bulk[int(tam/2):tam]
helpers.bulk(es,list_for_bulk,request_timeout=1000)
print("Send bulk")

hashs = Hash(64,24)
print("Clear Hash")



In [72]:
es = Elasticsearch(hosts=ip)

helpers.bulk(es,list_for_bulk,request_timeout=1000)


(4, [])

In [5]:
def set_value_in_hash(string_text,index,position):
    global hashs
    
    hashs.text_in_hashs[index][position]+=string_text
    
def generate_word(list_index,id,id_min):
    #id = 1
    return list(map(lambda pos_hash,index_hash: set_value_in_hash("{}_{} ".format(id,id_min),index_hash,pos_hash) if(pos_hash > 0) else None ,list_index,list(range(len(list_index)))))

In [22]:
import os

ns = 16
nd = 5
quality = 0

os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_index.txt  ../data-fingerprint/SD14/Finger_data/Vector_Index/ {} {} {}".format(ns,nd,quality))
os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_search.txt ../data-fingerprint/SD14/Finger_data/Vector_Search/ {} {} {}".format(ns,nd,quality))
       
print("Generate Vectors")

Generate Vectors


In [ ]:
%%time
import numpy as np
from textwrap import wrap
import os

ns = 16
nd = 5
quality = 20

#os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_index.txt  ../data-fingerprint/SD14/Finger_data/Vector_Index/ {} {} {}".format(ns,nd,quality))
#os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_search.txt ../data-fingerprint/SD14/Finger_data/Vector_Search/ {} {} {}".format(ns,nd,quality))
       
print("Generate Vectors")
    
    
es = Elasticsearch(hosts=ip)
hashs = Hash(96,16)
print("Create hash")
hashs_boolean = Hash_Boolean(96,16)
print("Create hash Boolean")

for x,i,files in os.walk("../data-fingerprint/SD14/Finger_data/Vector_Index/"):
    for ind,file in enumerate(files):
        if(True):#ind >=2000):
            if(ind %300 == 0 and ind != 0):
                list_for_bulk = convert_hash_to_bulk(hashs,hashs_boolean)
                print(len(list_for_bulk))
                tam = len(list_for_bulk)
                #l1 = list_for_bulk[0:10]
                #l2 = list_for_bulk[int(tam/2):tam]
                helpers.bulk(es,list_for_bulk,request_timeout=1000)
                print("Send bulk")

                hashs = Hash(96,16)
                print("Clear Hash")

              #  break

            print(ind+1,file)
            vector_data = sc.textFile(x + file)
            list_positions = vector_data.map(lambda x:x.replace(" ","")).map(lambda x:wrap(x,16)).map(lambda x: list(map(lambda x1: int(x1,2),x)) ).collect()
            id = int(file.split('.')[0])
            len_list = len(list_positions)
            print(len_list)
            list(map(generate_word, list_positions, np.repeat(id,len_list), list(range(len_list))))

            

list_for_bulk = convert_hash_to_bulk(hashs,hashs_boolean)
print(len(list_for_bulk))
tam = len(list_for_bulk)
#l1 = list_for_bulk[0:10]
#l2 = list_for_bulk[int(tam/2):tam]
helpers.bulk(es,list_for_bulk,request_timeout=1000)
print("Send bulk")

hashs = Hash(64,24)
print("Clear Hash")



Generate Vectors
Create hash
Create hash Boolean
1 1597.txt
200
2 2133.txt
89
3 1502.txt
124
4 1798.txt
100
5 2139.txt
144
6 1701.txt
140
7 1604.txt
102
8 1981.txt
123
9 1990.txt
92
10 2054.txt
110
11 2048.txt
126
12 1768.txt
87
13 1928.txt
161
14 1869.txt
78
15 1603.txt
100
16 1843.txt
90
17 2056.txt
143
18 1977.txt
147
19 1720.txt
61
20 1696.txt
118
21 1616.txt
119
22 1559.txt
91
23 1864.txt
96
24 1510.txt
84
25 1985.txt
89
26 1791.txt
101
27 1534.txt
173
28 1639.txt
137
29 2112.txt
172
30 1939.txt
134
31 1605.txt
77
32 1518.txt
115
33 2069.txt
115
34 1602.txt
74
35 2055.txt
128
36 1972.txt
20
37 1971.txt
171
38 1761.txt
141
39 1576.txt
130
40 2223.txt
107
41 1974.txt
133
42 2071.txt
229
43 1589.txt
97
44 1627.txt
103
45 1630.txt
99
46 1845.txt
68
47 1577.txt
112
48 1695.txt
49
49 1788.txt
145
50 1927.txt
113
51 1793.txt
92
52 1943.txt
125
53 2157.txt
190
54 1594.txt
173
55 2046.txt
142
56 1693.txt
123
57 1805.txt
100
58 1596.txt
207
59 1764.txt
76
60 1961.txt
79
61 1832.txt
133
62 1

In [16]:
import numpy as np
from textwrap import wrap
import os

In [17]:

def gendata1(list_pos_in_hash):
   # list_not_index = [  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
    #    13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
    #    26,  27,  28,  29,  30,  31,  43,  50, 173, 180, 32,  33,  34,  39,  40,  41,  42,  51,  52,  61, 162, 171, 172,
    #   181, 182, 183, 184, 189, 190, 191]
    #list_not_index=[0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 21,
     #               25, 86, 90,16, 20, 91, 95]
        
    list_not_index = [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 21,25,16,20]
  #  list_not_index= []
    for i,pos in enumerate(list_pos_in_hash):
        if(pos != 0 and i not in list_not_index):
            yield {"index": "hash{}".format(i) }
            yield {
                    "query": {"terms": {"_id":[str(pos)]}}} 
                            
                            
                            
                            
                            
                            
      

In [18]:
def realize_search(data_consult):
    global ip
    from elasticsearch import Elasticsearch
    es = Elasticsearch(hosts=ip)
    if(data_consult != []):
        data_response = es.msearch(body=data_consult)
       # print(data_response)
      #  print(data_response)
      #  return data_response
      #  return []
        return list(map(lambda x : x['hits']['hits'][0]['_source']['text'] if x['hits']['hits'] != [] else '', data_response['responses']))
    else:
        return []
    

In [19]:

import numpy as np

def func_count_byKey_return_max(x):
    if(x != []):
        import numpy as np
        x_,y_ = np.unique(x,return_counts=True)
        y_max = np.max(y_)
        return " ".join([ i[0].split('_')[0] for i in list(filter(lambda x: x[1] == y_max,list(map(lambda x,y:(x,y) ,x_,y_)))) ])
    else:
        return ""
def func_rank_elements(persons):
    persons_list, count_ = np.unique(persons.split(),return_counts=True)
    return persons_list[np.argsort(-1*count_)]


In [21]:
import datetime                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
pos_rank =[]
for x,i,files in os.walk("../data-fingerprint/SD14/Finger_data/Vector_Search/"):
    count = 0
    for file in files:
        
       # print(file)
        if(True):#count >= 618):
            time1 = datetime.datetime.now()
            vector_data = sc.textFile(x + file)
            persons = vector_data.map(lambda x:x.replace(" ","")).map(lambda x:wrap(x,16)).map(lambda x: list(map(lambda x1: int(x1,2),x))).map(lambda x : list(gendata1(x))).map(lambda x: realize_search(x)).map(lambda x : " ".join(x)).map(lambda x : x.split()).map(func_count_byKey_return_max ).reduce(lambda x,y: y + " "+ x).replace("  ", " ")
            ranking = func_rank_elements(persons)
            time2 = datetime.datetime.now()
            id = file.split(".")[0]
            pos_  = -1
            for p in range(len(ranking)):
                #print(ranking[p] ," a ")
                if(id  == ranking[p]):
                    pos_ = p + 1
           #         break
            print("count:",count,"filename:",file,"pos_rank:",pos_,"time:",time2-time1)
            pos_rank.append([pos_,file,time2-time1])
 #       break
        count+=1

        

count: 0 filename: 1597.txt pos_rank: 1 time: 0:01:06.643205


KeyboardInterrupt: 

In [41]:
import pandas as pd
df = pd.DataFrame(pos_rank,columns=['rank_pos','file','time'])
df.to_csv('result_new_method_secretaria.csv',index=False)

In [42]:
df[(df['rank_pos'] > 0) &(df['rank_pos'] <=52 )].shape

(856, 3)

In [43]:
1 - 848/1020

0.1686274509803921

In [44]:
1 - 438/1020

0.5705882352941176

In [46]:
1 - 856/1020

0.16078431372549018

In [3]:
from elasticsearch import Elasticsearch
import  pandas as pd
es = Elasticsearch(hosts=ip)
count_index_voids = []
for i in range(64):
    doc = {
        'size' : 10000,
        'query': {
            'match_all' : {}
       }
    }
    data_r = es.search(index='hash{}'.format(i), doc_type='pos', body=doc,scroll='1m')
    #data_r = es.get(index="hashs",id=i,doc_type="hash")
    count_void_elements = 0
    
    for partial_data in data_r['hits']['hits']:
            #if(partial_data['_source']['text'] == ''):
            count_void_elements +=1
      #      else:
       #         print(partial_data['_source']['text'])
        
    print(count_void_elements)
    scrollId = data_r['_scroll_id']
    scrollId_ant = ''
    while scrollId != scrollId_ant:
        print(scrollId, count_void_elements)
        data_r = es.scroll(scroll_id = scrollId, scroll = '1m')
        for partial_data in data_r['hits']['hits']:
            #if(partial_data['_source']['text'] == ''):
            count_void_elements +=1
            
        scrollId_ant = scrollId
        scrollId = data_r['_scroll_id']

         #   else:
        #        print(partial_data['_id'],partial_data['_index'])
                #print(partial_data['_source']['text'])
        
     #  ' if(count_void_elements len(data_r['_source']['index_in_hash']))
    count_index_voids.append([count_void_elements,i])
    print(i)
  #  break

/home/johnny/.local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


1
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAABFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAAhZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAAAMWVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAAAEFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAABRZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 1
0
1
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAAGFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAChZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAAAcWVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAAAIFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAACRZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 1
1
1
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAALFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAADBZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAAA0WVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAAAOFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAADxZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 1
2
1
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAAQFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAERZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAABMWVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAAASFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAFBZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 1
3
1
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAAVFlZJSTZFTDdQUl82NEExeklBSk

34
575
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAACwFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAshZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAALEWVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAACzFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAtBZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 575
35
596
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAC1FlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAthZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAALcWVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAAC4FlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAuRZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 596
36
1795
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAC6FlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAuxZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAALwWVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAAC9FlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAvhZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 1795
37
1803
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAC_FlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAwhZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAAMAWVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAADBFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAwxZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 1803
38
1866
DnF1ZXJ5VGhlbkZldGNoBQAAAAAA

In [9]:
import pandas as pd
import numpy as np

In [5]:
df = pd.DataFrame(count_index_voids,columns=['voids','index'])


In [6]:
df

,voids,index
0,1,0
1,1,1
2,1,2
3,1,3
4,1,4
...,...,...
59,1648,59
60,1876,60
61,1867,61
62,1822,62


In [7]:
df.describe()

,voids,index
count,64.000000,64.000000
mean,1050.328125,31.500000
std,864.959528,18.618987
min,0.000000,0.000000
25%,1.000000,15.750000
50%,1635.500000,31.500000
75%,1833.000000,47.250000
max,2018.000000,63.000000


In [10]:
np.unique(df.iloc[:,0],return_counts=True)

(array([   0,    1,   60,   62,  112,  117,  575,  587,  596,  619, 1340,
        1439, 1453, 1627, 1644, 1648, 1655, 1703, 1707, 1734, 1751, 1757,
        1768, 1795, 1803, 1809, 1818, 1822, 1825, 1831, 1839, 1852, 1866,
        1867, 1870, 1876, 1888, 1923, 1927, 1944, 1951, 1959, 1976, 2018]),
 array([ 2, 18,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,
         1,  1,  1,  2,  1,  1,  1,  1,  1,  1]))

In [11]:
df.median()

voids    1635.5
index      31.5
dtype: float64

### Existem 20 hashs em que todos os elementos caíram em 0 e foram eliminados

In [12]:
df[df['voids'] ==0].shape

(2, 2)

In [13]:
df[df['voids'] ==0]['index'].values

array([16, 20])

### Existem 36 hashs que todos os elementos estão na mesma posição, ou seja, são dados agrupados na mesma posição (muito volume) e são irrelevantes

In [14]:
df[df['voids'] ==1].shape

(18, 2)

In [15]:
df[df['voids'] ==1]['index'].values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 21,
       25])